This is where we turn the categorical data into numbers so that they can be used in regression. We did this by multi indexing first by a category, then by all movies a category is in. Then we can get the financial results of the previous movie and the average of all prior movies. This way we keep the model trained on past data when future data would be more available and easy to work with. There are no comments because it is so repetitive down there but the code just takes all inclusive values above it in its primary index and shift upwards filling in zero at the top.

In [ ]:
import pandas as pd
import numpy as np
imdb = pd.read_csv('Final3.csv')
pd.set_option('display.max_columns', None)

In [9]:
producer1list = []
producer2list = []
producer3list = []

for i in imdb['Producer']:
    try:
        p = i.split(',')
        producer1list.append(p[0])
        try:
            producer2list.append(p[1])
            try:
                producer3list.append(p[2])
            except:
                producer3list.append('unknown')
        except:
            producer2list.append('unknown')
            producer3list.append('unknown')
    except:
        producer1list.append('unknown')
        producer2list.append('unknown')
        producer3list.append('unknown')
        

In [11]:
imdb['Producer 1'] = producer1list
imdb['Producer 2'] = producer2list
imdb['Producer 3'] = producer3list

In [12]:
imdb

,IMDB Ratings,IMDB Score,Content Rating,Date Released,year,month,day of the week,Genres,Genre A,Genre B,Genre C,Keywords,Keyword 1,Keyword 2,Keyword 3,Stars,Star 1,Star 2,Star 3,Directors,Director 1,Director 2,Director 3,Writers,Writer 1,Writer 2,Writer 3,Runtime,Nominations,Awards,Estimated Budget,Budget Currency,Gross Revenue US&Canada USD,Opening Weekend Revenue US&Canada USD,Gross Revenue Worldwide USD,Name,RT_audience_score,RT_tomatometer_score,RT_audience_#reviews,RT_tomatometer_#reviews,Producer,Producer 1,Producer 2,Producer 3
0,469417,7.6,PG,2022-12-16,2022,12,4,['Action' 'Adventure' 'Fantasy'],Action,Adventure,Fantasy,['underwater' 'ocean' 'jungle' 'water' 'second...,underwater,ocean,jungle,['Sam Worthington' 'Zoe Saldana' 'Sigourney We...,Sam Worthington,Zoe Saldana,Sigourney Weaver,['James Cameron'],James Cameron,none,none,['James Cameron' 'Rick Jaffa' 'Amanda Silver'],James Cameron,Rick Jaffa,Amanda Silver,192,135,65,350000000,$,684075767,134100226,2320250281,Avatar: The Way of Water,8.6,7.1,48195.0,448.0,"James Cameron,Jon Landau",James Cameron,Jon Landau,unknown
1,161291,7.8,PG,2022-12-21,2022,12,2,['Animation' 'Adventure' 'Comedy'],Animation,Adventure,Comedy,['puss in boots character' 'sequel' 'cat' 'wol...,puss in boots character,sequel,cat,['Antonio Banderas' 'Salma Hayek' 'Harvey Guil...,Antonio Banderas,Salma Hayek,Harvey Guillén,['Joel Crawford' 'Januel Mercado'],Joel Crawford,Januel Mercado,none,['Paul Fisher' 'Tommy Swerdlow' 'Tom Wheeler'],Paul Fisher,Tommy Swerdlow,Tom Wheeler,102,56,5,90000000,$,185535345,12429515,481080374,Puss in Boots: The Last Wish,9.4,7.6,16575.0,192.0,Mark Swift,Mark Swift,unknown,unknown
2,153248,7.1,18A,2022-12-23,2022,12,4,['Comedy' 'Drama' 'History'],Comedy,Drama,History,['hollywood california' 'female full frontal n...,hollywood california,female full frontal nudity,1920s,['Brad Pitt' 'Margot Robbie' 'Jean Smart'],Brad Pitt,Margot Robbie,Jean Smart,['Damien Chazelle'],Damien Chazelle,none,none,['Damien Chazelle'],Damien Chazelle,none,none,189,146,43,110000000,$,15351455,3603368,63351455,Babylon,7.4,6.4,6999.0,358.0,"Marc Platt,Matthew Plouffe,Olivia Hamilton",Marc Platt,Matthew Plouffe,Olivia Hamilton
3,141943,7.5,G,2023-01-13,2023,1,4,['Comedy' 'Drama'],Comedy,Drama,none,['remake' 'suicide attempt' 'grumpy old man' '...,remake,suicide attempt,grumpy old man,['Tom Hanks' 'Mariana Treviño' 'Rachel Keller'],Tom Hanks,Mariana Treviño,Rachel Keller,['Marc Forster'],Marc Forster,none,none,['Fredrik Backman' 'Hannes Holm' 'David Magee'],Fredrik Backman,Hannes Holm,David Magee,126,8,0,50000000,$,64267657,56257,113190218,A Man Called Otto,9.0,6.1,10436.0,207.0,"Fredrik Wikström,Rita Wilson,Tom Hanks,Gary Go...",Fredrik Wikström,Rita Wilson,Tom Hanks
4,132495,6.4,14A,2023-01-06,2023,1,4,['Horror' 'Sci-Fi' 'Thriller'],Horror,Sci-Fi,Thriller,['artificial intelligence' 'orphaned girl' 'ro...,artificial intelligence,orphaned girl,robot sci fi,['Allison Williams' 'Violet McGraw' 'Ronny Chi...,Allison Williams,Violet McGraw,Ronny Chieng,['Gerard Johnstone'],Gerard Johnstone,none,none,['Akela Cooper' 'James Wan'],Akela Cooper,James Wan,none,102,2,1,12000000,$,95043350,30429860,179973454,M3GAN,7.6,7.2,11348.0,315.0,"Jason Blum,James Wan,Couper Samuelson",Jason Blum,James Wan,Couper Samuelson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040,116925,7.2,18+,1974-03-08,1974,3,4,['Crime' 'Drama' 'Thriller'],Crime,Drama,Thriller,['italian american' 'epilepsy' 'bar' 'street g...,italian american,epilepsy,bar,['Robert De Niro' 'Harvey Keitel' 'David Proval'],Robert De Niro,Harvey Keitel,David Proval,['Martin Scorsese'],Martin Scorsese,none,none,['Martin Scorsese' 'Mardik Martin'],Martin Scorsese,Mardik Martin,none,112,5,5,500000,$,32645,32645,59034,Mean Streets,8.0,8.8,55003.0,75.0,Jonathan T. Taplin,Jonathan T. Taplin,unknown,unknown
2041,96624,7.4,PG,1973-08-11,1

In [16]:
imdb = imdb[imdb['Gross Revenue Worldwide USD'].notnull()]
#imdb = imdb[imdb['Budget Currency']=='USD']
imdb['ROI domestic'] = (imdb['Gross Revenue US&Canada USD']-imdb['Budget Converted'])/imdb['Budget Converted']
imdb['Profit domestic'] = (imdb['Gross Revenue US&Canada USD']-imdb['Budget Converted'])
imdb['ROI int'] = (imdb['Gross Revenue Worldwide USD']-imdb['Budget Converted'])/imdb['Budget Converted']
imdb['Profit int'] = (imdb['Gross Revenue Worldwide USD']-imdb['Budget Converted'])

In [17]:
imdb = imdb.iloc[::-1]
imdb = imdb.reset_index(drop=True)

In [18]:
directorlist = pd.unique(imdb[['Director 1', 'Director 2']].values.ravel('K'))
multi_df_directors = pd.concat([imdb[(imdb['Director 1']==i)|(imdb['Director 2']==i)] for i in directorlist], keys=directorlist)

In [19]:
multi_df_directors['Average ROI Domestic for Director'] = multi_df_directors.groupby(level=0)['ROI domestic'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_directors['Average ROI International for Director'] = multi_df_directors.groupby(level=0)['ROI int'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_directors['Average Profit Domestic for Director'] = multi_df_directors.groupby(level=0)['Profit domestic'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_directors['Average Profit International for Director'] = multi_df_directors.groupby(level=0)['Profit int'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_directors['Average Revenue Domestic for Director'] = multi_df_directors.groupby(level=0)['Gross Revenue US&Canada USD'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_directors['Average Revenue International for Director'] = multi_df_directors.groupby(level=0)['Gross Revenue Worldwide USD'].expanding().mean().fillna(0).reset_index(level=0, drop=True)

In [21]:
shifted_d = multi_df_directors.groupby(level=0)['Average ROI Domestic for Director'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_directors['Average ROI Domestic for Director uninclusive'] = shifted_d

shifted_d = multi_df_directors.groupby(level=0)['Average ROI International for Director'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_directors['Average ROI International for Director uninclusive'] = shifted_d

shifted_d = multi_df_directors.groupby(level=0)['Average Profit Domestic for Director'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_directors['Average Profit Domestic for Director uninclusive'] = shifted_d

shifted_d = multi_df_directors.groupby(level=0)['Average Profit International for Director'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_directors['Average Profit International for Director uninclusive'] = shifted_d

shifted_d = multi_df_directors.groupby(level=0)['Average ROI Domestic for Director'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_directors['Average Revenue Domestic for Director uninclusive'] = shifted_d

shifted_d = multi_df_directors.groupby(level=0)['Average ROI International for Director'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_directors['Average Revenue International for Director uninclusive'] = shifted_d



shifted_d = multi_df_directors.groupby(level=0)['ROI domestic'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_directors['Previous ROI Domestic for Director'] = shifted_d

shifted_d = multi_df_directors.groupby(level=0)['ROI int'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_directors['Previous ROI International for Director'] = shifted_d

shifted_d = multi_df_directors.groupby(level=0)['Profit domestic'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_directors['Previous Profit Domestic for Director'] = shifted_d

shifted_d = multi_df_directors.groupby(level=0)['Profit int'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_directors['Previous Profit International for Director'] = shifted_d

shifted_d = multi_df_directors.groupby(level=0)['Gross Revenue Worldwide USD'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_directors['Previous Revenue Domestic for Director'] = shifted_d

shifted_d = multi_df_directors.groupby(level=0)['Gross Revenue US&Canada USD'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_directors['Previous Revenue International for Director'] = shifted_d

In [23]:
dir1droiave = []
dir1droiprev = []
dir1iroiave = []
dir1iroiprev = []

dir1drevave = []
dir1drevprev = []
dir1irevave = []
dir1irevprev = []

dir1dprofave = []
dir1dprofprev = []
dir1iprofave = []
dir1iprofprev = []

for idx,i in enumerate(imdb['Director 1']):
    a,b,c,d = multi_df_directors.loc[i].loc[idx][['Average ROI Domestic for Director uninclusive','Previous ROI Domestic for Director','Average ROI International for Director uninclusive','Previous ROI International for Director']]
    e,f,g,h = multi_df_directors.loc[i].loc[idx][['Average Revenue Domestic for Director uninclusive','Previous Revenue Domestic for Director','Average Revenue International for Director uninclusive','Previous Revenue International for Director']]
    i,j,k,l = multi_df_directors.loc[i].loc[idx][['Average Profit Domestic for Director uninclusive','Previous Profit Domestic for Director','Average Profit International for Director uninclusive','Previous Profit International for Director']]
    
    dir1droiave.append(a)
    dir1droiprev.append(b)
    dir1iroiave.append(c)
    dir1iroiprev.append(d)

    dir1drevave.append(e)
    dir1drevprev.append(f)
    dir1irevave.append(g)
    dir1irevprev.append(h)

    dir1dprofave.append(i)
    dir1dprofprev.append(j)
    dir1iprofave.append(k)
    dir1iprofprev.append(l)

In [24]:
imdb['dir1droiave'] = dir1droiave
imdb['dir1droiprev'] = dir1droiprev
imdb['dir1iroiave'] = dir1iroiave
imdb['dir1iroiprev'] = dir1iroiprev

imdb['dir1drevave'] = dir1drevave
imdb['dir1drevprev'] = dir1drevprev
imdb['dir1irevave'] = dir1irevave
imdb['dir1irevprev'] = dir1irevprev

imdb['dir1dprofave'] = dir1dprofave
imdb['dir1dprofprev'] = dir1dprofprev
imdb['dir1iprofave'] = dir1iprofave
imdb['dir1iprofprev'] = dir1iprofprev

In [25]:
dir2droiave = []
dir2droiprev = []
dir2iroiave = []
dir2iroiprev = []

dir2drevave = []
dir2drevprev = []
dir2irevave = []
dir2irevprev = []

dir2dprofave = []
dir2dprofprev = []
dir2iprofave = []
dir2iprofprev = []

for idx,i in enumerate(imdb['Director 2']):
    a,b,c,d = multi_df_directors.loc[i].loc[idx][['Average ROI Domestic for Director uninclusive','Previous ROI Domestic for Director','Average ROI International for Director uninclusive','Previous ROI International for Director']]
    e,f,g,h = multi_df_directors.loc[i].loc[idx][['Average Revenue Domestic for Director uninclusive','Previous Revenue Domestic for Director','Average Revenue International for Director uninclusive','Previous Revenue International for Director']]
    i,j,k,l = multi_df_directors.loc[i].loc[idx][['Average Profit Domestic for Director uninclusive','Previous Profit Domestic for Director','Average Profit International for Director uninclusive','Previous Profit International for Director']]
    
    dir2droiave.append(a)
    dir2droiprev.append(b)
    dir2iroiave.append(c)
    dir2iroiprev.append(d)

    dir2drevave.append(e)
    dir2drevprev.append(f)
    dir2irevave.append(g)
    dir2irevprev.append(h)

    dir2dprofave.append(i)
    dir2dprofprev.append(j)
    dir2iprofave.append(k)
    dir2iprofprev.append(l)

In [26]:
imdb['dir2droiave'] = dir2droiave
imdb['dir2droiprev'] = dir2droiprev
imdb['dir2iroiave'] = dir2iroiave
imdb['dir2iroiprev'] = dir2iroiprev

imdb['dir2drevave'] = dir2drevave
imdb['dir2drevprev'] = dir2drevprev
imdb['dir2irevave'] = dir2irevave
imdb['dir2irevprev'] = dir2irevprev

imdb['dir2dprofave'] = dir2dprofave
imdb['dir2dprofprev'] = dir2dprofprev
imdb['dir2iprofave'] = dir2iprofave
imdb['dir2iprofprev'] = dir2iprofprev

In [27]:
actorlist = pd.unique(imdb[['Star 1', 'Star 2', 'Star 3']].values.ravel('K'))
multi_df_actors = pd.concat([imdb[(imdb['Star 1']==i)|(imdb['Star 2']==i)|(imdb['Star 3']==i)] for i in actorlist], keys=actorlist)

In [28]:
multi_df_actors['Average ROI Domestic for Actor'] = multi_df_actors.groupby(level=0)['ROI domestic'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_actors['Average ROI International for Actor'] = multi_df_actors.groupby(level=0)['ROI int'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_actors['Average Profit Domestic for Actor'] = multi_df_actors.groupby(level=0)['Profit domestic'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_actors['Average Profit International for Actor'] = multi_df_actors.groupby(level=0)['Profit int'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_actors['Average Revenue Domestic for Actor'] = multi_df_actors.groupby(level=0)['Gross Revenue US&Canada USD'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_actors['Average Revenue International for Actor'] = multi_df_actors.groupby(level=0)['Gross Revenue Worldwide USD'].expanding().mean().fillna(0).reset_index(level=0, drop=True)

In [29]:
shifted_d = multi_df_actors.groupby(level=0)['Average ROI Domestic for Actor'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_actors['Average ROI Domestic for Actor uninclusive'] = shifted_d

shifted_d = multi_df_actors.groupby(level=0)['Average ROI International for Actor'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_actors['Average ROI International for Actor uninclusive'] = shifted_d

shifted_d = multi_df_actors.groupby(level=0)['Average Profit Domestic for Actor'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_actors['Average Profit Domestic for Actor uninclusive'] = shifted_d

shifted_d = multi_df_actors.groupby(level=0)['Average Profit International for Actor'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_actors['Average Profit International for Actor uninclusive'] = shifted_d

shifted_d = multi_df_actors.groupby(level=0)['Average ROI Domestic for Actor'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_actors['Average Revenue Domestic for Actor uninclusive'] = shifted_d

shifted_d = multi_df_actors.groupby(level=0)['Average ROI International for Actor'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_actors['Average Revenue International for Actor uninclusive'] = shifted_d



shifted_d = multi_df_actors.groupby(level=0)['ROI domestic'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_actors['Previous ROI Domestic for Actor'] = shifted_d

shifted_d = multi_df_actors.groupby(level=0)['ROI int'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_actors['Previous ROI International for Actor'] = shifted_d

shifted_d = multi_df_actors.groupby(level=0)['Profit domestic'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_actors['Previous Profit Domestic for Actor'] = shifted_d

shifted_d = multi_df_actors.groupby(level=0)['Profit int'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_actors['Previous Profit International for Actor'] = shifted_d

shifted_d = multi_df_actors.groupby(level=0)['Gross Revenue Worldwide USD'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_actors['Previous Revenue Domestic for Actor'] = shifted_d

shifted_d = multi_df_actors.groupby(level=0)['Gross Revenue US&Canada USD'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_actors['Previous Revenue International for Actor'] = shifted_d

In [30]:
star1droiave = []
star1droiprev = []
star1iroiave = []
star1iroiprev = []

star1drevave = []
star1drevprev = []
star1irevave = []
star1irevprev = []

star1dprofave = []
star1dprofprev = []
star1iprofave = []
star1iprofprev = []

for idx,i in enumerate(imdb['Star 1']):
    a,b,c,d = multi_df_actors.loc[i].loc[idx][['Average ROI Domestic for Actor uninclusive','Previous ROI Domestic for Actor','Average ROI International for Actor uninclusive','Previous ROI International for Actor']]
    e,f,g,h = multi_df_actors.loc[i].loc[idx][['Average Revenue Domestic for Actor uninclusive','Previous Revenue Domestic for Actor','Average Revenue International for Actor uninclusive','Previous Revenue International for Actor']]
    i,j,k,l = multi_df_actors.loc[i].loc[idx][['Average Profit Domestic for Actor uninclusive','Previous Profit Domestic for Actor','Average Profit International for Actor uninclusive','Previous Profit International for Actor']]
    
    star1droiave.append(a)
    star1droiprev.append(b)
    star1iroiave.append(c)
    star1iroiprev.append(d)

    star1drevave.append(e)
    star1drevprev.append(f)
    star1irevave.append(g)
    star1irevprev.append(h)

    star1dprofave.append(i)
    star1dprofprev.append(j)
    star1iprofave.append(k)
    star1iprofprev.append(l)

In [31]:
imdb['star1droiave'] = star1droiave
imdb['star1droiprev'] = star1droiprev
imdb['star1iroiave'] = star1iroiave
imdb['star1iroiprev'] = star1iroiprev

imdb['star1drevave'] = star1drevave
imdb['star1drevprev'] = star1drevprev
imdb['star1irevave'] = star1irevave
imdb['star1irevprev'] = star1irevprev

imdb['star1dprofave'] = star1dprofave
imdb['star1dprofprev'] = star1dprofprev
imdb['star1iprofave'] = star1iprofave
imdb['star1iprofprev'] = star1iprofprev

In [32]:
star2droiave = []
star2droiprev = []
star2iroiave = []
star2iroiprev = []

star2drevave = []
star2drevprev = []
star2irevave = []
star2irevprev = []

star2dprofave = []
star2dprofprev = []
star2iprofave = []
star2iprofprev = []

for idx,i in enumerate(imdb['Star 2']):
    
    if i != 'unknown':
    
        a,b,c,d = multi_df_actors.loc[i].loc[idx][['Average ROI Domestic for Actor uninclusive','Previous ROI Domestic for Actor','Average ROI International for Actor uninclusive','Previous ROI International for Actor']]
        e,f,g,h = multi_df_actors.loc[i].loc[idx][['Average Revenue Domestic for Actor uninclusive','Previous Revenue Domestic for Actor','Average Revenue International for Actor uninclusive','Previous Revenue International for Actor']]
        i,j,k,l = multi_df_actors.loc[i].loc[idx][['Average Profit Domestic for Actor uninclusive','Previous Profit Domestic for Actor','Average Profit International for Actor uninclusive','Previous Profit International for Actor']]

        star2droiave.append(a)
        star2droiprev.append(b)
        star2iroiave.append(c)
        star2iroiprev.append(d)

        star2drevave.append(e)
        star2drevprev.append(f)
        star2irevave.append(g)
        star2irevprev.append(h)

        star2dprofave.append(i)
        star2dprofprev.append(j)
        star2iprofave.append(k)
        star2iprofprev.append(l)
        
    else:
        
        star2droiave.append(np.nan)
        star2droiprev.append(np.nan)
        star2iroiave.append(np.nan)
        star2iroiprev.append(np.nan)

        star2drevave.append(np.nan)
        star2drevprev.append(np.nan)
        star2irevave.append(np.nan)
        star2irevprev.append(np.nan)

        star2dprofave.append(np.nan)
        star2dprofprev.append(np.nan)
        star2iprofave.append(np.nan)
        star2iprofprev.append(np.nan)
        
        

In [33]:
imdb['star2droiave'] = star2droiave
imdb['star2droiprev'] = star2droiprev
imdb['star2iroiave'] = star2iroiave
imdb['star2iroiprev'] = star2iroiprev

imdb['star2drevave'] = star2drevave
imdb['star2drevprev'] = star2drevprev
imdb['star2irevave'] = star2irevave
imdb['star2irevprev'] = star2irevprev

imdb['star2dprofave'] = star2dprofave
imdb['star2dprofprev'] = star2dprofprev
imdb['star2iprofave'] = star2iprofave
imdb['star2iprofprev'] = star2iprofprev

In [34]:
star3droiave = []
star3droiprev = []
star3iroiave = []
star3iroiprev = []

star3drevave = []
star3drevprev = []
star3irevave = []
star3irevprev = []

star3dprofave = []
star3dprofprev = []
star3iprofave = []
star3iprofprev = []

for idx,i in enumerate(imdb['Star 3']):
    
    if i != 'unknown':
    
        a,b,c,d = multi_df_actors.loc[i].loc[idx][['Average ROI Domestic for Actor uninclusive','Previous ROI Domestic for Actor','Average ROI International for Actor uninclusive','Previous ROI International for Actor']]
        e,f,g,h = multi_df_actors.loc[i].loc[idx][['Average Revenue Domestic for Actor uninclusive','Previous Revenue Domestic for Actor','Average Revenue International for Actor uninclusive','Previous Revenue International for Actor']]
        i,j,k,l = multi_df_actors.loc[i].loc[idx][['Average Profit Domestic for Actor uninclusive','Previous Profit Domestic for Actor','Average Profit International for Actor uninclusive','Previous Profit International for Actor']]

        star3droiave.append(a)
        star3droiprev.append(b)
        star3iroiave.append(c)
        star3iroiprev.append(d)

        star3drevave.append(e)
        star3drevprev.append(f)
        star3irevave.append(g)
        star3irevprev.append(h)

        star3dprofave.append(i)
        star3dprofprev.append(j)
        star3iprofave.append(k)
        star3iprofprev.append(l)
        
    else:
        
        star3droiave.append(np.nan)
        star3droiprev.append(np.nan)
        star3iroiave.append(np.nan)
        star3iroiprev.append(np.nan)

        star3drevave.append(np.nan)
        star3drevprev.append(np.nan)
        star3irevave.append(np.nan)
        star3irevprev.append(np.nan)

        star3dprofave.append(np.nan)
        star3dprofprev.append(np.nan)
        star3iprofave.append(np.nan)
        star3iprofprev.append(np.nan)
        
        

In [35]:
imdb['star3droiave'] = star3droiave
imdb['star3droiprev'] = star3droiprev
imdb['star3iroiave'] = star3iroiave
imdb['star3iroiprev'] = star3iroiprev

imdb['star3drevave'] = star3drevave
imdb['star3drevprev'] = star3drevprev
imdb['star3irevave'] = star3irevave
imdb['star3irevprev'] = star3irevprev

imdb['star3dprofave'] = star3dprofave
imdb['star3dprofprev'] = star3dprofprev
imdb['star3iprofave'] = star3iprofave
imdb['star3iprofprev'] = star3iprofprev

In [36]:
writerlist = pd.unique(imdb[['Writer 1', 'Writer 2', 'Writer 3']].values.ravel('K'))
multi_df_writers = pd.concat([imdb[(imdb['Writer 1']==i)|(imdb['Writer 2']==i)|(imdb['Writer 3']==i)] for i in writerlist], keys=writerlist)

In [37]:
multi_df_writers['Average ROI Domestic for Writer'] = multi_df_writers.groupby(level=0)['ROI domestic'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_writers['Average ROI International for Writer'] = multi_df_writers.groupby(level=0)['ROI int'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_writers['Average Profit Domestic for Writer'] = multi_df_writers.groupby(level=0)['Profit domestic'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_writers['Average Profit International for Writer'] = multi_df_writers.groupby(level=0)['Profit int'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_writers['Average Revenue Domestic for Writer'] = multi_df_writers.groupby(level=0)['Gross Revenue US&Canada USD'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_writers['Average Revenue International for Writer'] = multi_df_writers.groupby(level=0)['Gross Revenue Worldwide USD'].expanding().mean().fillna(0).reset_index(level=0, drop=True)

In [38]:
shifted_d = multi_df_writers.groupby(level=0)['Average ROI Domestic for Writer'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_writers['Average ROI Domestic for Writer uninclusive'] = shifted_d

shifted_d = multi_df_writers.groupby(level=0)['Average ROI International for Writer'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_writers['Average ROI International for Writer uninclusive'] = shifted_d

shifted_d = multi_df_writers.groupby(level=0)['Average Profit Domestic for Writer'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_writers['Average Profit Domestic for Writer uninclusive'] = shifted_d

shifted_d = multi_df_writers.groupby(level=0)['Average Profit International for Writer'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_writers['Average Profit International for Writer uninclusive'] = shifted_d

shifted_d = multi_df_writers.groupby(level=0)['Average ROI Domestic for Writer'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_writers['Average Revenue Domestic for Writer uninclusive'] = shifted_d

shifted_d = multi_df_writers.groupby(level=0)['Average ROI International for Writer'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_writers['Average Revenue International for Writer uninclusive'] = shifted_d



shifted_d = multi_df_writers.groupby(level=0)['ROI domestic'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_writers['Previous ROI Domestic for Writer'] = shifted_d

shifted_d = multi_df_writers.groupby(level=0)['ROI int'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_writers['Previous ROI International for Writer'] = shifted_d

shifted_d = multi_df_writers.groupby(level=0)['Profit domestic'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_writers['Previous Profit Domestic for Writer'] = shifted_d

shifted_d = multi_df_writers.groupby(level=0)['Profit int'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_writers['Previous Profit International for Writer'] = shifted_d

shifted_d = multi_df_writers.groupby(level=0)['Gross Revenue Worldwide USD'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_writers['Previous Revenue Domestic for Writer'] = shifted_d

shifted_d = multi_df_writers.groupby(level=0)['Gross Revenue US&Canada USD'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_writers['Previous Revenue International for Writer'] = shifted_d

In [39]:
writer1droiave = []
writer1droiprev = []
writer1iroiave = []
writer1iroiprev = []

writer1drevave = []
writer1drevprev = []
writer1irevave = []
writer1irevprev = []

writer1dprofave = []
writer1dprofprev = []
writer1iprofave = []
writer1iprofprev = []

for idx,i in enumerate(imdb['Writer 1']):
    a,b,c,d = multi_df_writers.loc[i].loc[idx][['Average ROI Domestic for Writer uninclusive','Previous ROI Domestic for Writer','Average ROI International for Writer uninclusive','Previous ROI International for Writer']]
    e,f,g,h = multi_df_writers.loc[i].loc[idx][['Average Revenue Domestic for Writer uninclusive','Previous Revenue Domestic for Writer','Average Revenue International for Writer uninclusive','Previous Revenue International for Writer']]
    i,j,k,l = multi_df_writers.loc[i].loc[idx][['Average Profit Domestic for Writer uninclusive','Previous Profit Domestic for Writer','Average Profit International for Writer uninclusive','Previous Profit International for Writer']]
    
    writer1droiave.append(a)
    writer1droiprev.append(b)
    writer1iroiave.append(c)
    writer1iroiprev.append(d)

    writer1drevave.append(e)
    writer1drevprev.append(f)
    writer1irevave.append(g)
    writer1irevprev.append(h)

    writer1dprofave.append(i)
    writer1dprofprev.append(j)
    writer1iprofave.append(k)
    writer1iprofprev.append(l)

In [41]:
imdb['writer1droiave'] = writer1droiave
imdb['writer1droiprev'] = writer1droiprev
imdb['writer1iroiave'] = writer1iroiave
imdb['writer1iroiprev'] = writer1iroiprev

imdb['writer1drevave'] = writer1drevave
imdb['writer1drevprev'] = writer1drevprev
imdb['writer1irevave'] = writer1irevave
imdb['writer1irevprev'] = writer1irevprev

imdb['writer1dprofave'] = writer1dprofave
imdb['writer1dprofprev'] = writer1dprofprev
imdb['writer1iprofave'] = writer1iprofave
imdb['writer1iprofprev'] = writer1iprofprev

In [42]:
writer2droiave = []
writer2droiprev = []
writer2iroiave = []
writer2iroiprev = []

writer2drevave = []
writer2drevprev = []
writer2irevave = []
writer2irevprev = []

writer2dprofave = []
writer2dprofprev = []
writer2iprofave = []
writer2iprofprev = []

for idx,i in enumerate(imdb['Writer 2']):
    a,b,c,d = multi_df_writers.loc[i].loc[idx][['Average ROI Domestic for Writer uninclusive','Previous ROI Domestic for Writer','Average ROI International for Writer uninclusive','Previous ROI International for Writer']]
    e,f,g,h = multi_df_writers.loc[i].loc[idx][['Average Revenue Domestic for Writer uninclusive','Previous Revenue Domestic for Writer','Average Revenue International for Writer uninclusive','Previous Revenue International for Writer']]
    i,j,k,l = multi_df_writers.loc[i].loc[idx][['Average Profit Domestic for Writer uninclusive','Previous Profit Domestic for Writer','Average Profit International for Writer uninclusive','Previous Profit International for Writer']]
    
    writer2droiave.append(a)
    writer2droiprev.append(b)
    writer2iroiave.append(c)
    writer2iroiprev.append(d)

    writer2drevave.append(e)
    writer2drevprev.append(f)
    writer2irevave.append(g)
    writer2irevprev.append(h)

    writer2dprofave.append(i)
    writer2dprofprev.append(j)
    writer2iprofave.append(k)
    writer2iprofprev.append(l)

In [44]:
imdb['writer2droiave'] = writer2droiave
imdb['writer2droiprev'] = writer2droiprev
imdb['writer2iroiave'] = writer2iroiave
imdb['writer2iroiprev'] = writer2iroiprev

imdb['writer2drevave'] = writer2drevave
imdb['writer2drevprev'] = writer2drevprev
imdb['writer2irevave'] = writer2irevave
imdb['writer2irevprev'] = writer2irevprev

imdb['writer2dprofave'] = writer2dprofave
imdb['writer2dprofprev'] = writer2dprofprev
imdb['writer2iprofave'] = writer2iprofave
imdb['writer2iprofprev'] = writer2iprofprev

In [45]:
writer3droiave = []
writer3droiprev = []
writer3iroiave = []
writer3iroiprev = []

writer3drevave = []
writer3drevprev = []
writer3irevave = []
writer3irevprev = []

writer3dprofave = []
writer3dprofprev = []
writer3iprofave = []
writer3iprofprev = []

for idx,i in enumerate(imdb['Writer 3']):
    a,b,c,d = multi_df_writers.loc[i].loc[idx][['Average ROI Domestic for Writer uninclusive','Previous ROI Domestic for Writer','Average ROI International for Writer uninclusive','Previous ROI International for Writer']]
    e,f,g,h = multi_df_writers.loc[i].loc[idx][['Average Revenue Domestic for Writer uninclusive','Previous Revenue Domestic for Writer','Average Revenue International for Writer uninclusive','Previous Revenue International for Writer']]
    i,j,k,l = multi_df_writers.loc[i].loc[idx][['Average Profit Domestic for Writer uninclusive','Previous Profit Domestic for Writer','Average Profit International for Writer uninclusive','Previous Profit International for Writer']]
    
    writer3droiave.append(a)
    writer3droiprev.append(b)
    writer3iroiave.append(c)
    writer3iroiprev.append(d)

    writer3drevave.append(e)
    writer3drevprev.append(f)
    writer3irevave.append(g)
    writer3irevprev.append(h)

    writer3dprofave.append(i)
    writer3dprofprev.append(j)
    writer3iprofave.append(k)
    writer3iprofprev.append(l)

In [47]:
imdb['writer3droiave'] = writer3droiave
imdb['writer3droiprev'] = writer3droiprev
imdb['writer3iroiave'] = writer3iroiave
imdb['writer3iroiprev'] = writer3iroiprev

imdb['writer3drevave'] = writer3drevave
imdb['writer3drevprev'] = writer3drevprev
imdb['writer3irevave'] = writer3irevave
imdb['writer3irevprev'] = writer3irevprev

imdb['writer3dprofave'] = writer3dprofave
imdb['writer3dprofprev'] = writer3dprofprev
imdb['writer3iprofave'] = writer3iprofave
imdb['writer3iprofprev'] = writer3iprofprev

In [48]:
keylist = pd.unique(imdb[['Keyword 1', 'Keyword 2', 'Keyword 3', 'Keyword 4', 'Keyword 5']].values.ravel('K'))

In [49]:
multi_df_keys = pd.concat([imdb[(imdb['Keyword 1']==i)|(imdb['Keyword 2']==i)|(imdb['Keyword 3']==i)|(imdb['Keyword 4']==i)|(imdb['Keyword 5']==i)] for i in keylist], keys=keylist)

In [50]:
multi_df_keys['Average ROI Domestic for Key'] = multi_df_keys.groupby(level=0)['ROI domestic'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_keys['Average ROI International for Key'] = multi_df_keys.groupby(level=0)['ROI int'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_keys['Average Profit Domestic for Key'] = multi_df_keys.groupby(level=0)['Profit domestic'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_keys['Average Profit International for Key'] = multi_df_keys.groupby(level=0)['Profit int'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_keys['Average Revenue Domestic for Key'] = multi_df_keys.groupby(level=0)['Gross Revenue US&Canada USD'].expanding().mean().fillna(0).reset_index(level=0, drop=True)
multi_df_keys['Average Revenue International for Key'] = multi_df_keys.groupby(level=0)['Gross Revenue Worldwide USD'].expanding().mean().fillna(0).reset_index(level=0, drop=True)

In [53]:
shifted_d = multi_df_keys.groupby(level=0)['Average ROI Domestic for Key'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_keys['Average ROI Domestic for Key uninclusive'] = shifted_d

shifted_d = multi_df_keys.groupby(level=0)['Average ROI International for Key'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_keys['Average ROI International for Key uninclusive'] = shifted_d

shifted_d = multi_df_keys.groupby(level=0)['Average Profit Domestic for Key'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_keys['Average Profit Domestic for Key uninclusive'] = shifted_d

shifted_d = multi_df_keys.groupby(level=0)['Average Profit International for Key'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_keys['Average Profit International for Key uninclusive'] = shifted_d

shifted_d = multi_df_keys.groupby(level=0)['Average ROI Domestic for Key'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_keys['Average Revenue Domestic for Key uninclusive'] = shifted_d

shifted_d = multi_df_keys.groupby(level=0)['Average ROI International for Key'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_keys['Average Revenue International for Key uninclusive'] = shifted_d



shifted_d = multi_df_keys.groupby(level=0)['ROI domestic'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_keys['Previous ROI Domestic for Key'] = shifted_d

shifted_d = multi_df_keys.groupby(level=0)['ROI int'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_keys['Previous ROI International for Key'] = shifted_d

shifted_d = multi_df_keys.groupby(level=0)['Profit domestic'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_keys['Previous Profit Domestic for Key'] = shifted_d

shifted_d = multi_df_keys.groupby(level=0)['Profit int'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_keys['Previous Profit International for Key'] = shifted_d

shifted_d = multi_df_keys.groupby(level=0)['Gross Revenue Worldwide USD'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_keys['Previous Revenue Domestic for Key'] = shifted_d

shifted_d = multi_df_keys.groupby(level=0)['Gross Revenue US&Canada USD'].transform(lambda x: x.shift(1)).fillna(0)
multi_df_keys['Previous Revenue International for Key'] = shifted_d

In [54]:
key1droiave = []
key1droiprev = []
key1iroiave = []
key1iroiprev = []

key1drevave = []
key1drevprev = []
key1irevave = []
key1irevprev = []

key1dprofave = []
key1dprofprev = []
key1iprofave = []
key1iprofprev = []

for idx,i in enumerate(imdb['Keyword 1']):
    a,b,c,d = multi_df_keys.loc[i].loc[idx][['Average ROI Domestic for Key uninclusive','Previous ROI Domestic for Key','Average ROI International for Key uninclusive','Previous ROI International for Key']]
    e,f,g,h = multi_df_keys.loc[i].loc[idx][['Average Revenue Domestic for Key uninclusive','Previous Revenue Domestic for Key','Average Revenue International for Key uninclusive','Previous Revenue International for Key']]
    i,j,k,l = multi_df_keys.loc[i].loc[idx][['Average Profit Domestic for Key uninclusive','Previous Profit Domestic for Key','Average Profit International for Key uninclusive','Previous Profit International for Key']]
    
    key1droiave.append(a)
    key1droiprev.append(b)
    key1iroiave.append(c)
    key1iroiprev.append(d)

    key1drevave.append(e)
    key1drevprev.append(f)
    key1irevave.append(g)
    key1irevprev.append(h)

    key1dprofave.append(i)
    key1dprofprev.append(j)
    key1iprofave.append(k)
    key1iprofprev.append(l)

In [55]:
imdb['key1droiave'] = key1droiave
imdb['key1droiprev'] = key1droiprev
imdb['key1iroiave'] = key1iroiave
imdb['key1iroiprev'] = key1iroiprev

imdb['key1drevave'] = key1drevave
imdb['key1drevprev'] = key1drevprev
imdb['key1irevave'] = key1irevave
imdb['key1irevprev'] = key1irevprev

imdb['key1dprofave'] = key1dprofave
imdb['key1dprofprev'] = key1dprofprev
imdb['key1iprofave'] = key1iprofave
imdb['key1iprofprev'] = key1iprofprev

In [56]:
key2droiave = []
key2droiprev = []
key2iroiave = []
key2iroiprev = []

key2drevave = []
key2drevprev = []
key2irevave = []
key2irevprev = []

key2dprofave = []
key2dprofprev = []
key2iprofave = []
key2iprofprev = []

for idx,i in enumerate(imdb['Keyword 2']):
    a,b,c,d = multi_df_keys.loc[i].loc[idx][['Average ROI Domestic for Key uninclusive','Previous ROI Domestic for Key','Average ROI International for Key uninclusive','Previous ROI International for Key']]
    e,f,g,h = multi_df_keys.loc[i].loc[idx][['Average Revenue Domestic for Key uninclusive','Previous Revenue Domestic for Key','Average Revenue International for Key uninclusive','Previous Revenue International for Key']]
    i,j,k,l = multi_df_keys.loc[i].loc[idx][['Average Profit Domestic for Key uninclusive','Previous Profit Domestic for Key','Average Profit International for Key uninclusive','Previous Profit International for Key']]
    
    key2droiave.append(a)
    key2droiprev.append(b)
    key2iroiave.append(c)
    key2iroiprev.append(d)

    key2drevave.append(e)
    key2drevprev.append(f)
    key2irevave.append(g)
    key2irevprev.append(h)

    key2dprofave.append(i)
    key2dprofprev.append(j)
    key2iprofave.append(k)
    key2iprofprev.append(l)

In [57]:
imdb['key2droiave'] = key2droiave
imdb['key2droiprev'] = key2droiprev
imdb['key2iroiave'] = key2iroiave
imdb['key2iroiprev'] = key2iroiprev

imdb['key2drevave'] = key2drevave
imdb['key2drevprev'] = key2drevprev
imdb['key2irevave'] = key2irevave
imdb['key2irevprev'] = key2irevprev

imdb['key2dprofave'] = key2dprofave
imdb['key2dprofprev'] = key2dprofprev
imdb['key2iprofave'] = key2iprofave
imdb['key2iprofprev'] = key2iprofprev

In [58]:
key3droiave = []
key3droiprev = []
key3iroiave = []
key3iroiprev = []

key3drevave = []
key3drevprev = []
key3irevave = []
key3irevprev = []

key3dprofave = []
key3dprofprev = []
key3iprofave = []
key3iprofprev = []

for idx,i in enumerate(imdb['Keyword 3']):
    a,b,c,d = multi_df_keys.loc[i].loc[idx][['Average ROI Domestic for Key uninclusive','Previous ROI Domestic for Key','Average ROI International for Key uninclusive','Previous ROI International for Key']]
    e,f,g,h = multi_df_keys.loc[i].loc[idx][['Average Revenue Domestic for Key uninclusive','Previous Revenue Domestic for Key','Average Revenue International for Key uninclusive','Previous Revenue International for Key']]
    i,j,k,l = multi_df_keys.loc[i].loc[idx][['Average Profit Domestic for Key uninclusive','Previous Profit Domestic for Key','Average Profit International for Key uninclusive','Previous Profit International for Key']]
    
    key3droiave.append(a)
    key3droiprev.append(b)
    key3iroiave.append(c)
    key3iroiprev.append(d)

    key3drevave.append(e)
    key3drevprev.append(f)
    key3irevave.append(g)
    key3irevprev.append(h)

    key3dprofave.append(i)
    key3dprofprev.append(j)
    key3iprofave.append(k)
    key3iprofprev.append(l)

In [59]:
imdb['key3droiave'] = key3droiave
imdb['key3droiprev'] = key3droiprev
imdb['key3iroiave'] = key3iroiave
imdb['key3iroiprev'] = key3iroiprev

imdb['key3drevave'] = key3drevave
imdb['key3drevprev'] = key3drevprev
imdb['key3irevave'] = key3irevave
imdb['key3irevprev'] = key3irevprev

imdb['key3dprofave'] = key3dprofave
imdb['key3dprofprev'] = key3dprofprev
imdb['key3iprofave'] = key3iprofave
imdb['key3iprofprev'] = key3iprofprev

In [60]:
key4droiave = []
key4droiprev = []
key4iroiave = []
key4iroiprev = []

key4drevave = []
key4drevprev = []
key4irevave = []
key4irevprev = []

key4dprofave = []
key4dprofprev = []
key4iprofave = []
key4iprofprev = []

for idx,i in enumerate(imdb['Keyword 3']):
    a,b,c,d = multi_df_keys.loc[i].loc[idx][['Average ROI Domestic for Key uninclusive','Previous ROI Domestic for Key','Average ROI International for Key uninclusive','Previous ROI International for Key']]
    e,f,g,h = multi_df_keys.loc[i].loc[idx][['Average Revenue Domestic for Key uninclusive','Previous Revenue Domestic for Key','Average Revenue International for Key uninclusive','Previous Revenue International for Key']]
    i,j,k,l = multi_df_keys.loc[i].loc[idx][['Average Profit Domestic for Key uninclusive','Previous Profit Domestic for Key','Average Profit International for Key uninclusive','Previous Profit International for Key']]
    
    key4droiave.append(a)
    key4droiprev.append(b)
    key4iroiave.append(c)
    key4iroiprev.append(d)

    key4drevave.append(e)
    key4drevprev.append(f)
    key4irevave.append(g)
    key4irevprev.append(h)

    key4dprofave.append(i)
    key4dprofprev.append(j)
    key4iprofave.append(k)
    key4iprofprev.append(l)

In [61]:
imdb['key4droiave'] = key4droiave
imdb['key4droiprev'] = key4droiprev
imdb['key4iroiave'] = key4iroiave
imdb['key4iroiprev'] = key4iroiprev

imdb['key4drevave'] = key4drevave
imdb['key4drevprev'] = key4drevprev
imdb['key4irevave'] = key4irevave
imdb['key4irevprev'] = key4irevprev

imdb['key4dprofave'] = key4dprofave
imdb['key4dprofprev'] = key4dprofprev
imdb['key4iprofave'] = key4iprofave
imdb['key4iprofprev'] = key4iprofprev

In [62]:
key5droiave = []
key5droiprev = []
key5iroiave = []
key5iroiprev = []

key5drevave = []
key5drevprev = []
key5irevave = []
key5irevprev = []

key5dprofave = []
key5dprofprev = []
key5iprofave = []
key5iprofprev = []

for idx,i in enumerate(imdb['Keyword 3']):
    a,b,c,d = multi_df_keys.loc[i].loc[idx][['Average ROI Domestic for Key uninclusive','Previous ROI Domestic for Key','Average ROI International for Key uninclusive','Previous ROI International for Key']]
    e,f,g,h = multi_df_keys.loc[i].loc[idx][['Average Revenue Domestic for Key uninclusive','Previous Revenue Domestic for Key','Average Revenue International for Key uninclusive','Previous Revenue International for Key']]
    i,j,k,l = multi_df_keys.loc[i].loc[idx][['Average Profit Domestic for Key uninclusive','Previous Profit Domestic for Key','Average Profit International for Key uninclusive','Previous Profit International for Key']]
    
    key5droiave.append(a)
    key5droiprev.append(b)
    key5iroiave.append(c)
    key5iroiprev.append(d)

    key5drevave.append(e)
    key5drevprev.append(f)
    key5irevave.append(g)
    key5irevprev.append(h)

    key5dprofave.append(i)
    key5dprofprev.append(j)
    key5iprofave.append(k)
    key5iprofprev.append(l)

In [63]:
imdb['key5droiave'] = key5droiave
imdb['key5droiprev'] = key5droiprev
imdb['key5iroiave'] = key5iroiave
imdb['key5iroiprev'] = key5iroiprev

imdb['key5drevave'] = key5drevave
imdb['key5drevprev'] = key5drevprev
imdb['key5irevave'] = key5irevave
imdb['key5irevprev'] = key5irevprev

imdb['key5dprofave'] = key5dprofave
imdb['key5dprofprev'] = key5dprofprev
imdb['key5iprofave'] = key5iprofave
imdb['key5iprofprev'] = key5iprofprev

In [66]:
imdb.to_csv('Final5.csv', index=False)  # Save the updated DataFrame to a CSV file

In [64]:
imdb

,IMDB Ratings,IMDB Score,Content Rating,Date Released,Genre 1,Genre 2,Genre 3,Keyword 1,Keyword 2,Keyword 3,Keyword 4,Keyword 5,Production Companies,Star 1,Star 1 url,Star 2,Star 2 url,Star 3,Star 3 url,Director 1,Director 1 url,Director 2,Director 2 url,Director 3 url,Writer 1,Writer 1 url,Writer 2,Writer 2 url,Writer 3,Writer 3 url,Runtime,Nominations,Awards,Gross Revenue US&Canada USD,Opening Weekend Revenue US&Canada USD,Gross Revenue Worldwide USD,year,month,day of the week,Budget Converted,inflation_adjustment,Name,RT_audience_score,RT_tomatometer_score,RT_audience_#reviews,RT_tomatometer_#reviews,ROI domestic,Profit domestic,ROI int,Profit int,dir1droiave,dir1droiprev,dir1iroiave,dir1iroiprev,dir1drevave,dir1drevprev,dir1irevave,dir1irevprev,dir1dprofave,dir1dprofprev,dir1iprofave,dir1iprofprev,dir2droiave,dir2droiprev,dir2iroiave,dir2iroiprev,dir2drevave,dir2drevprev,dir2irevave,dir2irevprev,dir2dprofave,dir2dprofprev,dir2iprofave,dir2iprofprev,star1droiave,star1droiprev,star1iroiave,star1iroiprev,star1drevave,star1drevprev,star1irevave,star1irevprev,star1dprofave,star1dprofprev,star1iprofave,star1iprofprev,star2droiave,star2droiprev,star2iroiave,star2iroiprev,star2drevave,star2drevprev,star2irevave,star2irevprev,star2dprofave,star2dprofprev,star2iprofave,star2iprofprev,star3droiave,star3droiprev,star3iroiave,star3iroiprev,star3drevave,star3drevprev,star3irevave,star3irevprev,star3dprofave,star3dprofprev,star3iprofave,star3iprofprev,writer1droiave,writer1droiprev,writer1iroiave,writer1iroiprev,writer1drevave,writer1drevprev,writer1irevave,writer1irevprev,writer1dprofave,writer1dprofprev,writer1iprofave,writer1iprofprev,writer2droiave,writer2droiprev,writer2iroiave,writer2iroiprev,writer2drevave,writer2drevprev,writer2irevave,writer2irevprev,writer2dprofave,writer2dprofprev,writer2iprofave,writer2iprofprev,writer3droiave,writer3droiprev,writer3iroiave,writer3iroiprev,writer3drevave,writer3drevprev,writer3irevave,writer3irevprev,writer3dprofave,writer3dprofprev,writer3iprofave,writer3iprofprev,key1droiave,key1droiprev,key1iroiave,key1iroiprev,key1drevave,key1drevprev,key1irevave,key1irevprev,key1dprofave,key1dprofprev,key1iprofave,key1iprofprev,key2droiave,key2droiprev,key2iroiave,key2iroiprev,key2drevave,key2drevprev,key2irevave,key2irevprev,key2dprofave,key2dprofprev,key2iprofave,key2iprofprev,key3droiave,key3droiprev,key3iroiave,key3iroiprev,key3drevave,key3drevprev,key3irevave,key3irevprev,key3dprofave,key3dprofprev,key3iprofave,key3iprofprev,key4droiave,key4droiprev,key4iroiave,key4iroiprev,key4drevave,key4drevprev,key4irevave,key4irevprev,key4dprofave,key4dprofprev,key4iprofave,key4iprofprev,key5droiave,key5droiprev,key5iroiave,key5iroiprev,key5drevave,key5drevprev,key5irevave,key5irevprev,key5dprofave,key5dprofprev,key5iprofave,key5iprofprev
0,6217.0,6.2,PG,2002-08-23,Comedy,Drama,Romance,sex offender,female sex offender,female sex abuser,matronizing,unpunished sex crime,"['Miramax', 'InDigEnt (Independent Digital Ent...",Sigourney Weaver,https://www.imdb.com/name/nm0000244/,Aaron Stanford,https://www.imdb.com/name/nm0822155/,Kate Mara,https://www.imdb.com/name/nm0544718/,Gary Winick,https://www.imdb.com/name/nm0935095/,unknown,unknown,unknown,Heather McGowan,https://www.imdb.com/name/nm0569641/,Niels Mueller,https://www.imdb.com/name/nm0007158/,Gary Winick,https://www.imdb.com/name/nm0935095/,78,4,3,5.378647e+06,80682.0,5.953391e+06,2002,8,4,2.790442e+05,1.860295,Tadpole,3.4,6.7,5154.0,108.0,18.275253,5.099603e+06,20.334940,5.674347e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e